In [241]:
import numpy as np
import scipy.special as sp
from numpy import linalg as al
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import csv

from itertools import combinations
from random import random , choice

import winsound


#-------------------------------------------------------

#funciones varias

def graph_from_csv(filename):
    df = pd.read_csv(filename, index_col=0)
    return nx.from_pandas_adjacency(df)

def positiv(graph):
    for i in graph.nodes:
        graph[i][i]['weight'] = 0
    for j in graph.edges:
        if graph[j[0]][j[1]]['weight']<0:
            graph[j[0]][j[1]]['weight'] = -graph[j[0]][j[1]]['weight'] 
    return(graph)

def random_weighted_graph(the_nodes):
    G = nx.Graph()
    G.add_nodes_from(the_nodes)
    all_edges = combinations(the_nodes, 2)
    elist = [(e[0], e[1], random()) for e in all_edges]
    G.add_weighted_edges_from(elist)
    return G


def eigenvalores(A):
        av=list(al.eigh(A))
    
        av[0]=av[0].astype(float)
        av[1]=av[1].astype(float)
    
        for i in range(len(av[0]-1)):
            av[0][i]=round(float(av[0][i]),5)
    
        return av[0]
    
def eigenvectores(A):
        av=list(al.eigh(A))
    
        av[1]=av[1].astype(float)
    
        for i in range(len(av[1]-1)):
            for j in range(len(av[1][i]-1)):
                av[1][i][j]=round(float(av[1][i][j]),3)
    
        return av[1].T
    
def Fiedler(G):
        lap=nx.laplacian_matrix(G)
        print("valor Fiedler")
        print(eigenvalores(lap.A)[1])
        print("vector Fiedler")
        print(eigenvectores(lap.A)[1])
        print("los eigenvalores")
        print(eigenvalores(lap.A))
    
    # Función genérica para el cociente de Rayleigh para una matriz y un vector,
    
def rayleigh(A,v):
        numerador = np.dot(np.dot(A,v),v)
        denominador = np.dot(v,v)
        S = numerador/denominador
        return S
    
def peso_grafica(G):
    long=len(G.nodes)
    peso=sum([list(G.degree(weight='weight'))[k][1]/2 for k in range(long)])
    return(peso)
    
    
def vector_candidato(G): #un vector sencillo candidato a tener cociente de Rayleigh pequeño
    grados=[list(G.degree(weight='weight'))[k][1] for k in range(len(G))]

    nodo_uno=list(G.nodes)[np.argmin(np.array(grados))]

    grados1=[list(G.subgraph(set(G.nodes)-{nodo_uno}).degree(weight='weight'))[k][1] for k in range(len(G)-1)]
    #e2=np.where(np.array(grados1)==min(grados1))
    nodo_dos=list(G.nodes)[np.argmin(np.array(grados1))]

    v=[0]*len(G)
    v[np.argmin(np.array(grados))]=1
    v[np.argmin(np.array(grados1))]=-1
    return(v)

def es_completa(G): #revisa si una gráfica es completa; falla si hay nodos conectados a sí mismos
    es = True
    for i in G.nodes:
        if G.degree[i]<len(G.nodes)-1:
            es = False
            break
    return(es)       



def otro_maximo_local(G,pasos=11,m=11):
    #m = int(np.ceil(len(G.nodes)/2))
    grados = np.array([list(G.degree(weight='weight'))[k][1] for k in range(len(G.nodes))])
    nuevos_nodos = list(np.array(G.nodes)[np.argsort(grados)[-m:]])
    optima = G.subgraph(nuevos_nodos)
    subgraf = optima
    maximal_ac = conectividad_algebraica(subgraf)
    for i in range(pasos):
        if len(subgraf.nodes)==len(G.nodes) or len(subgraf.nodes)==1:
            break
        tres_graficas=[quitar_uno_express(subgraf),optima,poner_uno_express(G,subgraf)]
        dos_graficas=[quitar_uno_express(subgraf),poner_uno_express(G,subgraf)]
        tres_conectividades=np.array([conectividad_algebraica(k) for k in tres_graficas])
        dos_conectividades=np.array([conectividad_algebraica(k) for k in dos_graficas])
        optima=tres_graficas[np.argmax(tres_conectividades)]
        subgraf=dos_graficas[np.argmax(dos_conectividades)]
        
    return(optima)
    
        
def pares_vetados(G,cota): #pares de nodos que no pueden estar en subgráfica con valor Fiedler mayor a cota
    lap = nx.laplacian_matrix(G)
    pares = [tuple([aristas[0],aristas[1]]) for aristas in G.edges if aristas[0]!=aristas[1]]
    vetados = []
    for pareja in pares:
        v = [0]*len(G.nodes)
        for k in range(len(G.nodes)):
            if list(G.nodes)[k]==pareja[0]:
                v[k]=-1
            if list(G.nodes)[k]==pareja[1]:
                v[k]=1
        cociente = rayleigh(lap.A,v)
        if cociente<cota:
            vetados.append(pareja)
    return(vetados)
    

def subgrafica_optima_ac(G):#Dada una gráfica con pesos, regresa la subgráfica con máximo valor Fiedler
    optimal=otro_maximo_local(G)
    print("Máximo local en subgrafica con nodos", list(optimal.nodes))
    print("Longitud",len(optimal.nodes))
    print("Valor Fiedler es", conectividad_algebraica(optimal))
    cota = conectividad_algebraica(otro_maximo_local(G))
    vets = pares_vetados(G,cota)
    print("Número de pares vetados es",len(vets))
    G_sp = nx.complete_graph(list(G.nodes))
    G_sp.remove_edges_from(vets)
    
    maximal_ac=cota
    for k in range(1,len(G.nodes)-int(np.ceil(cota))+1):
        size = len(G.nodes)-k
        print("\rArmando lista ", size, "nodos",end="")
        por_checar=[G.subgraph(nodos) for nodos in list(combinations(G.nodes,size)) if es_completa(G_sp.subgraph(nodos))] 
        if len(por_checar)>0:
            print("\rChecando",len(por_checar),"subgráficas de tamaño", size)
        for subgraph in por_checar:
            if conectividad_algebraica(subgraph)>maximal_ac:
                maximal_ac=conectividad_algebraica(subgraph)
                optimal=subgraph
                print("\rNuevo máximo es ",maximal_ac,end="")
    print("El máximo valor Fiedler global se encuentra en",list(optimal.nodes))
    print("Longitud",len(optimal.nodes))
    print("Valor Fiedler máximo es",maximal_ac)
    return(optimal)
    
    return(G.subgraph(nodos_selectos))    
    

#--------------------------------------------------------    

#funciones particulares para explorar valores Fiedler de subgráficas de los datos del PSG

def quitar_uno_express(S): #regresa la subgrafica con un nodo menos y mayor Fiedler
    todos = tuple(S.nodes)
    reducidos = [conectividad_algebraica(S.subgraph(set(todos)-{nodo})) for nodo in todos]
    weak_link={todos[np.argmax(reducidos)]}
    nuevos_nodos = tuple(set(todos)-weak_link)  
    
    return(S.subgraph(nuevos_nodos))

def poner_uno_express(G,S): #regresa la subgrafica con mayor Fiedler de entre las gráficas S más otro nodo
    sobrantes=tuple(set(G.nodes)-set(S.nodes))
    graficas_aumentadas = [G.subgraph(tuple(set.union(set(S.nodes),{nodo_extra}))) for nodo_extra in sobrantes]
    vf_aumentadas = [conectividad_algebraica(aumentadas) for aumentadas in graficas_aumentadas]
    optimo = np.argmax(vf_aumentadas)
    
    return(graficas_aumentadas[optimo])    
    

def conectividad_algebraica(G):
    laplace=nx.laplacian_matrix(G)
    return(eigenvalores(laplace.A)[1])

In [224]:
all_nodes = ['C3', 'C4', 'CZ', 'EMG', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'LOG', 'O1', 'O2', 'P3', 'P4', 'PZ', 
             'ROG', 'T3', 'T4', 'T5', 'T6']

ac_tocho=[]
max_locales=[]
cuantos_nodos=[]

for i in range(10):
    G=random_weighted_graph(all_nodes)
    ML=otro_maximo_local(G)
    ac_tocho.append(conectividad_algebraica(G))
    ML=otro_maximo_local(G)
    max_locales.append(conectividad_algebraica(ML))
    cuantos_nodos.append(len(ML.nodes))

print("Fiedler value promedio (de toda la gráfica):",round(np.mean(ac_tocho),3))
print("Del máximo local",round(np.mean(max_locales),3))
print("nodos promedio del OML:",np.mean(cuantos_nodos))

Fiedler value promedio (de toda la gráfica): 7.483
Del máximo local 7.66
nodos promedio del OML: 20.4


In [226]:
all_nodes = ['C3', 'C4', 'CZ', 'EMG', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'LOG', 'O1', 'O2', 'P3', 'P4', 'PZ', 
             'ROG', 'T3', 'T4', 'T5', 'T6']

G=random_weighted_graph(all_nodes)
subgrafica_optima_ac(G)

Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'EMG', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'LOG', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 21
Valor Fiedler es 7.38028
Número de pares vetados es 0
Checando 22 subgráficas de tamaño 21
Checando 231 subgráficas de tamaño 20
Checando 1540 subgráficas de tamaño 19
Checando 7315 subgráficas de tamaño 18
Checando 26334 subgráficas de tamaño 17
Checando 74613 subgráficas de tamaño 16
Checando 170544 subgráficas de tamaño 15
Checando 319770 subgráficas de tamaño 14
Checando 497420 subgráficas de tamaño 13
Checando 646646 subgráficas de tamaño 12
Checando 705432 subgráficas de tamaño 11
Checando 646646 subgráficas de tamaño 10
Checando 497420 subgráficas de tamaño 9
Checando 319770 subgráficas de tamaño 8
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'EMG', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'LOG', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 21
Valor Fiedler máx

In [178]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_AEFP.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_AEFP=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.00503
Máximo local en subgrafica con nodos ['C4', 'CZ', 'F3', 'F4', 'F7', 'FZ', 'O1', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 13
Valor Fiedler es 11.36844
Número de pares vetados es 114
Checando 1 subgráficas de tamaño 15
Checando 15 subgráficas de tamaño 14
Checando 106 subgráficas de tamaño 13
Checando 467 subgráficas de tamaño 12
El máximo valor Fiedler global se encuentra en ['C4', 'CZ', 'F3', 'F4', 'F7', 'FZ', 'O1', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 13
Valor Fiedler máximo es 11.36844


In [3]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_CLMN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_AEFP=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 3.20352
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FZ', 'P3', 'T3', 'T5', 'T6']
Longitud 13
Valor Fiedler es 8.73162
Número de pares vetados es 22
Checando 1 subgráficas de tamaño 19
Checandolista  18 nodos 20 subgráficas de tamaño 18
Checando 188 subgráficas de tamaño 17
Checando 1105 subgráficas de tamaño 16
Checando 4556 subgráficas de tamaño 15
Checando 14008 subgráficas de tamaño 14
Checando 33320 subgráficas de tamaño 13
Checando 62764 subgráficas de tamaño 12
Checando 95030 subgráficas de tamaño 11
Checando 116688 subgráficas de tamaño 10
Checando 116688 subgráficas de tamaño 9
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FZ', 'P3', 'T3', 'T5', 'T6']
Longitud 13
Valor Fiedler máximo es 8.73162


In [179]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_EMNN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_EMNN=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.49411
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T4']
Longitud 15
Valor Fiedler es 8.6096
Número de pares vetados es 67
Checando 1 subgráficas de tamaño 17
Checando 18 subgráficas de tamaño 16
Checando 151 subgráficas de tamaño 15
Checando 786 subgráficas de tamaño 14
Checando 2848 subgráficas de tamaño 13
Checando 7631 subgráficas de tamaño 12
Checando 15665 subgráficas de tamaño 11
Checando 25168 subgráficas de tamaño 10
Checando 32032 subgráficas de tamaño 9
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T4']
Longitud 15
Valor Fiedler máximo es 8.6096


In [227]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_FGH.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_FGH=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.11089
Máximo local en subgrafica con nodos ['PZ', 'O1', 'C3', 'T4', 'LOG', 'T3', 'C4', 'T5', 'T6']
Longitud 9
Valor Fiedler es 5.84814
Número de pares vetados es 122
Checando 1 subgráficas de tamaño 13
Checando 13 subgráficas de tamaño 12
Checando 78 subgráficas de tamaño 11
Checando 287 subgráficas de tamaño 10
Checando 724 subgráficas de tamaño 9
Checando 1323 subgráficas de tamaño 8
Checando 1801 subgráficas de tamaño 7
Checando 1848 subgráficas de tamaño 6
El máximo valor Fiedler global se encuentra en ['PZ', 'O1', 'C3', 'T4', 'LOG', 'T3', 'C4', 'T5', 'T6']
Longitud 9
Valor Fiedler máximo es 5.84814


In [235]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_GH.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_GH=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.00479
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F7', 'F8', 'FP2', 'FZ', 'LOG', 'O1', 'O2', 'P3', 'P4', 'PZ', 'ROG', 'T5', 'T6']
Longitud 16
Valor Fiedler es 11.21374
Número de pares vetados es 69
Checando 2 subgráficas de tamaño 17
Checando 33 subgráficas de tamaño 16
Checando 256 subgráficas de tamaño 15
Checando 1240 subgráficas de tamaño 14
Checando 4200 subgráficas de tamaño 13
Checando 10556 subgráficas de tamaño 12
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F7', 'F8', 'FP2', 'FZ', 'LOG', 'O1', 'O2', 'P3', 'P4', 'PZ', 'ROG', 'T5', 'T6']
Longitud 16
Valor Fiedler máximo es 11.21374


In [242]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_GURM.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_GURM=subgrafica_optima_ac(G)


Valor Fiedler de la gráfica es 0.69036
Máximo local en subgrafica con nodos ['PZ', 'O1', 'C3', 'T4', 'T3', 'CZ', 'P3', 'C4', 'T5', 'T6']
Longitud 10
Valor Fiedler es 4.34986
Número de pares vetados es 44
Checando 1 subgráficas de tamaño 16
Checando 18 subgráficas de tamaño 15
Checando 148 subgráficas de tamaño 14
Checando 743 subgráficas de tamaño 13
Checando 2560 subgráficas de tamaño 12
Checando 6438 subgráficas de tamaño 11
Checando 12252 subgráficas de tamaño 10
Checando 18031 subgráficas de tamaño 9
Checando 20767 subgráficas de tamaño 8
Checando 18797 subgráficas de tamaño 7
Checando 13329 subgráficas de tamaño 6
Checando 7320 subgráficas de tamaño 5
El máximo valor Fiedler global se encuentra en ['PZ', 'O1', 'C3', 'CZ', 'P3', 'C4', 'T5', 'T6']
Longitud 8
Valor Fiedler máximo es 6.15203


In [243]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_JANA.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_JANA=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 5.17703
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 17
Valor Fiedler es 11.42175
Número de pares vetados es 21
Checando 1 subgráficas de tamaño 21
Checando 21 subgráficas de tamaño 20
Checando 210 subgráficas de tamaño 19
Checando 1330 subgráficas de tamaño 18
Checando 5985 subgráficas de tamaño 17
Checando 20349 subgráficas de tamaño 16
Checando 54264 subgráficas de tamaño 15
Checando 116280 subgráficas de tamaño 14
Checando 203490 subgráficas de tamaño 13
Checando 293930 subgráficas de tamaño 12
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 19
Valor Fiedler máximo es 11.43821


In [187]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_JGMN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_JGMN=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 1.59509
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'O1', 'O2', 'P3', 'P4', 'PZ']
Longitud 13
Valor Fiedler es 11.51337
Número de pares vetados es 109
Checando 1 subgráficas de tamaño 16
Checando 16 subgráficas de tamaño 15
Checando 120 subgráficas de tamaño 14
Checando 560 subgráficas de tamaño 13
Checando 1820 subgráficas de tamaño 12
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'O1', 'O2', 'P3', 'P4', 'PZ']
Longitud 13
Valor Fiedler máximo es 11.51337


In [250]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_MGNA.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_MGNA=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.59396
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 18
Valor Fiedler es 8.50098
Número de pares vetados es 69
Checando 1 subgráficas de tamaño 18
Checando 18 subgráficas de tamaño 17
Checando 153 subgráficas de tamaño 16
Checando 816 subgráficas de tamaño 15
Checando 3060 subgráficas de tamaño 14
Checando 8568 subgráficas de tamaño 13
Checando 18564 subgráficas de tamaño 12
Checando 31824 subgráficas de tamaño 11
Checando 43758 subgráficas de tamaño 10
Checando 48620 subgráficas de tamaño 9
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 18
Valor Fiedler máximo es 8.50098


In [245]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_MJNN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_MJNN=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 1.55748
Máximo local en subgrafica con nodos ['C3', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FZ', 'O2', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 14
Valor Fiedler es 10.19833
Número de pares vetados es 109
Checando 1 subgráficas de tamaño 16
Checando 16 subgráficas de tamaño 15
Checando 120 subgráficas de tamaño 14
Checando 560 subgráficas de tamaño 13
Checando 1820 subgráficas de tamaño 12
Checando 4368 subgráficas de tamaño 11
El máximo valor Fiedler global se encuentra en ['C3', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FZ', 'O2', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
Longitud 14
Valor Fiedler máximo es 10.19833


In [249]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_PCM.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_PCM=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 1.25291
Máximo local en subgrafica con nodos ['C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FZ', 'O2', 'PZ', 'T3', 'T4']
Longitud 12
Valor Fiedler es 7.25326
Número de pares vetados es 85
Checando 1 subgráficas de tamaño 17
Checando 17 subgráficas de tamaño 16
Checando 136 subgráficas de tamaño 15
Checando 680 subgráficas de tamaño 14
Checando 2380 subgráficas de tamaño 13
Checando 6188 subgráficas de tamaño 12
Checando 12376 subgráficas de tamaño 11
Checando 19448 subgráficas de tamaño 10
Checando 24311 subgráficas de tamaño 9
Checando 24318 subgráficas de tamaño 8
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FZ', 'O2', 'PZ', 'T3', 'T4']
Longitud 12
Valor Fiedler máximo es 7.25326


In [246]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_RLMN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_RLMN=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.52372
Máximo local en subgrafica con nodos ['C3', 'C4', 'F3', 'F4', 'F7', 'F8', 'FZ', 'LOG', 'O2', 'P3', 'PZ', 'ROG', 'T3', 'T4']
Longitud 14
Valor Fiedler es 5.02406
Número de pares vetados es 61
Checando 1 subgráficas de tamaño 16
Checando 16 subgráficas de tamaño 15
Checando 120 subgráficas de tamaño 14
Checando 560 subgráficas de tamaño 13
Checando 1820 subgráficas de tamaño 12
Checando 4368 subgráficas de tamaño 11
Checando 8008 subgráficas de tamaño 10
Checando 11440 subgráficas de tamaño 9
Checando 12870 subgráficas de tamaño 8
Checando 11441 subgráficas de tamaño 7
Checando 8014 subgráficas de tamaño 6
El máximo valor Fiedler global se encuentra en ['C3', 'C4', 'F3', 'F4', 'F7', 'F8', 'FZ', 'LOG', 'O2', 'P3', 'PZ', 'ROG', 'T3', 'T4']
Longitud 14
Valor Fiedler máximo es 5.02406


In [247]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_RRMN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_RRMN=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 0.11445
Máximo local en subgrafica con nodos ['PZ', 'O1', 'T4', 'P4', 'CZ', 'O2', 'P3', 'T5', 'T6']
Longitud 9
Valor Fiedler es 3.80565
Número de pares vetados es 57
Checando 1 subgráficas de tamaño 14
Checando 17 subgráficas de tamaño 13
Checando 127 subgráficas de tamaño 12
Checando 563 subgráficas de tamaño 11
Checando 1672 subgráficas de tamaño 10
Checando 3541 subgráficas de tamaño 9
Checando 5535 subgráficas de tamaño 8
Checando 6498 subgráficas de tamaño 7
Checando 5758 subgráficas de tamaño 6
Checando 3828 subgráficas de tamaño 5
Checando 1875 subgráficas de tamaño 4
El máximo valor Fiedler global se encuentra en ['PZ', 'O1', 'P4', 'O2', 'T6']
Longitud 5
Valor Fiedler máximo es 4.21676


In [248]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesNs/correls_VCNN.csv'))
print("Valor Fiedler de la gráfica es",conectividad_algebraica(G))
optima_VCNN=subgrafica_optima_ac(G)

Valor Fiedler de la gráfica es 1.63849
Máximo local en subgrafica con nodos ['C4', 'CZ', 'F8', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T4', 'T5', 'T6']
Longitud 11
Valor Fiedler es 5.48322
Número de pares vetados es 52
Checando 1 subgráficas de tamaño 16
Checando 17 subgráficas de tamaño 15
Checando 134 subgráficas de tamaño 14
Checando 653 subgráficas de tamaño 13
Checando 2208 subgráficas de tamaño 12
Checando 5501 subgráficas de tamaño 11
Checando 10450 subgráficas de tamaño 10
Checando 15433 subgráficas de tamaño 9
Checando 17889 subgráficas de tamaño 8
Checando 16312 subgráficas de tamaño 7
Checando 11657 subgráficas de tamaño 6
El máximo valor Fiedler global se encuentra en ['C4', 'CZ', 'F8', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T4', 'T5', 'T6']
Longitud 11
Valor Fiedler máximo es 5.48322
